# Data Preprocessing Tools

## Importing the libraries

In [254]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd


## Importing the maintenance dataset

In [255]:
# Load Excel files
df_maintenance = pd.read_csv("0_maintenance_raw_data/maintenance.csv", sep=";")

In [256]:
print(df_maintenance.columns)

Index(['lifetime', 'broken', 'pressureInd', 'moistureInd', 'temperatureInd',
       'team', 'provider'],
      dtype='object')


In [257]:
print(df_maintenance.head())

   lifetime  broken  pressureInd  moistureInd  temperatureInd   team  \
0        56       0    92.178854   104.230204       96.517159  TeamA   
1        81       1    72.075938   103.065701       87.271062  TeamC   
2        60       0    96.272254    77.801376      112.196170  TeamA   
3        86       1    94.406461   108.493608       72.025374  TeamC   
4        34       0    97.752899    99.413492      103.756271  TeamB   

    provider  
0  Provider4  
1  Provider4  
2  Provider1  
3  Provider2  
4  Provider1  


In [258]:
print(df_maintenance.shape)

(1000, 7)


## Importing service now dataset

In [259]:
# Load Excel files
df_service = pd.read_csv("0_maintenance_raw_data/service_now_only.csv")

In [260]:
# Selective renaming for df
df_service = df_service.rename(columns={
    'Asset Number': 'Asset_Number'
})


In [261]:
print(df_service.columns)

Index(['Asset_Number', 'Incident_Number', 'Short_Description', 'Status',
       'Priority', 'Category', 'Subcategory', 'Assigned_To', 'Assigned_Group',
       'Created_Date'],
      dtype='object')


In [262]:
print(df_service.head())

   Asset_Number Incident_Number           Short_Description   Status  \
0        100080      INC0010047    Access badge not working   Closed   
1        100081      INC0010085   Data center cooling alert   Closed   
2        300006      INC0010048   Printer toner replacement     Open   
3        300007      INC0010032  VoIP phone not registering  Stalled   
4        300008      INC0010058      Network switch failure  Stalled   

       Priority    Category      Subcategory   Assigned_To   Assigned_Group  \
0  3   Moderate    Security  Physical Access  Chris Parker    Security Team   
1  1   Critical  Facilities      Data Center  Michael Wong       Facilities   
2       4   Low    Hardware          Printer    Unassigned  Desktop Support   
3  3   Moderate    Hardware            Phone  Sarah Miller  Telecom Support   
4  1   Critical     Network           Switch  Michael Wong     Network Team   

          Created_Date  
0  2024 04 09T15:58:22  
1  2024 04 17T08:05:22  
2  2024 04 10T08:

In [263]:
print(df_service.shape)

(111, 10)


## Insert Asset Number in `df_maintenance` from `df_service`

In [264]:
unique_assets = df_service["Asset_Number"].unique()
repeated_assets = np.resize(unique_assets, df_maintenance.shape[0])

df_maintenance_with_asset_num = df_maintenance.copy()
df_maintenance_with_asset_num.insert(0, "Asset_Number", repeated_assets)

df_sorted_asc = df_maintenance_with_asset_num.sort_values(by='Asset_Number')


In [265]:
print(df_sorted_asc.head)

<bound method NDFrame.head of      Asset_Number  lifetime  broken  pressureInd  moistureInd  temperatureInd  \
0          100080        56       0    92.178854   104.230204       96.517159   
999        100080        63       0   116.901354    99.998694       47.641493   
666        100080        10       0   114.684324   112.865138       76.556266   
888        100080        60       1   120.351288    93.771542      114.251655   
333        100080        45       0   146.482610    98.848252      106.979033   
..            ...       ...     ...          ...          ...             ...   
554        600127        80       1   136.363399    89.902048       77.376099   
776        600127        31       0   118.393599    94.383096      110.938194   
221        600127        65       1   104.641411   111.418840       96.511544   
110        600127        92       1    99.961582   105.944788      114.997360   
998        600127        78       0   104.911649   104.257296       83.421491  

## Indexing

In [266]:
df_with_index = df_sorted_asc.copy()
df_with_index.insert(0, 'Observation_Index', range(len(df_sorted_asc)))

## Insert event date in `df_maintenance`

In [267]:
from datetime import timedelta

# Ensure Created_Date is in datetime format
df_service["Created_Date"] = pd.to_datetime(df_service["Created_Date"], format="%Y %m %dT%H:%M:%S").dt.date

# use sorted df_maintenance
df_with_index_1 = df_with_index.copy()

# Create a new column for observation date
observation_dates = []

# Process each asset group
for asset, group in df_with_index_1.groupby("Asset_Number"):
    # Get initial date from df_service
    start_date = df_service.loc[df_service["Asset_Number"] == asset, "Created_Date"].values[0]
    
    dates = []
    current_date = pd.to_datetime(start_date)
    
    for i, row in group.iterrows():
        dates.append(current_date)
        # Backtrack for next row using this row's lifetime
        current_date = current_date - timedelta(days=row["lifetime"])
    
    observation_dates.extend(dates)

# Assign observation_date column
df_with_index_1["observation_date"] = observation_dates


## Export Updated Excel

In [268]:
# Create the folder if it doesn't exist
os.makedirs("1_maintenance_survival_data", exist_ok=True)

# Save the Excel file inside the folder
df_with_index_1.to_csv("1_maintenance_survival_data/1_maintenance_survival_data.csv", index=False)